In [1]:
import os
import pandas as pd
import pydicom
from tqdm import tqdm

In [2]:
algo_output = r'D:\dataset\B78_out'
output_file = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\innovation_path\pilot_study\pilot_study_mapping_score.xlsx'

In [3]:
New_LoS = []
for folder in os.listdir(algo_output):
    if folder.startswith('P'):
        case = folder
        flag = False
        for root, _, files in os.walk(os.path.join(algo_output, folder)):
            for file in files:
                if file == 'caseLevelLoS.txt':
                    with open(os.path.join(root, file), 'r') as f:
                        los = float(f.read()[:-1])
                    New_LoS.append(los)
                    flag = True
                    break
            if flag:
                break
df = pd.read_excel(output_file)
df['LoS'] = New_LoS
df.to_excel(output_file, index=False)

In [4]:
reader_root = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\innovation_path\pilot_study\readers_annotation'

In [8]:
case_list = ['P' + (3 - len(str(x)))*'0' +str(x) for x in range(1,181)]
reader_list = ['KQQ','LF','WHC','YJY','ZHY','WL','WZ']
begin_data = []
begin_data.append('NR: 8\n')
begin_data.append('N0: 92\n')
begin_data.append('N1: 88\n')
begin_data.append('NM: 3\n')

begin_data.append('\n')

begin_data.append('BEGIN DATA:\n')

mapping_dict = {'reader_1_annotation.xlsx':'Unaided', 'reader_2_annotation.xlsx':'Aided', 'KQQ': 'Reader1', 'LF':'Reader2', 'WHC':'Reader3', 'YJY':'Reader4', 'ZHY': 'Reader5', 'WL': 'Reader6', 'WZ': 'Reader7'}
pathology_result = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\innovation_path\pilot_study\pilot_study_mapping_score.xlsx'

gt_df = pd.read_excel(pathology_result)
isup_threshold = 2
for index, row in gt_df.iterrows():
    cur_case = row['PID']
    cur_isup = row['ISUP']
    if cur_isup >= isup_threshold:
        cur_gt = 1
    else:
        cur_gt = 0
    begin_data.append(','.join(['-1', cur_case, '0', str(cur_gt)]) +'\n')

for index, row in gt_df.iterrows():
    cur_case = row['PID']
    cur_reader = 'AI'
    cur_los = row['LoS']
    cur_modality = 'AI_Standalone'
    begin_data.append(','.join([cur_reader, cur_case, cur_modality, str(cur_los)]) +'\n')
    
reader_results =['reader_1_annotation.xlsx', 'reader_2_annotation.xlsx']
for result in reader_results:
    reader_result = os.path.join(reader_root, result)
    reader_df = pd.read_excel(reader_result)
    for reader in reader_list:
        for index, row in reader_df.iterrows():
            cur_case = row['PID']
            cur_reader = mapping_dict.get(reader)
            cur_modality = mapping_dict.get(result)
            cur_los = str(row[reader+'_S'])
            begin_data.append(','.join([cur_reader, cur_case, cur_modality, cur_los]) + '\n') 
print(len(begin_data))
with open(f'raw_isup_data_isup_{isup_threshold}.csv', 'w') as f:
    f.writelines(begin_data)

2886
